In [27]:
# postprocess prodigy results by assigning label to each word in huggingface dataset format

In [1]:
import json
import spacy

from prodigy.components.db import connect

In [2]:
# assign dict with numbers for each entity component
entities = ['O', 'B-ACT', 'I-ACT', 'B-ATTR', 'I-ATTR', 'B-RELOP', 'I-RELOP', 'B-QUANT', 'I-QUANT', 'B-ENT', 'I-ENT']
entities_dict  = {entities[i]: i for i in range(len(entities))}

In [8]:
prodigy_name = 'ner_req_test_labeled'

In [9]:
db = connect()
prodigy_annotations = db.get_dataset(prodigy_name)
examples = ((eg["text"], eg) for eg in prodigy_annotations)
nlp = spacy.blank("en")

dataset = []
unique_id = 0 #train starts at 0, val at 636, test at 774 (137+636)
for doc, eg in nlp.pipe(examples, as_tuples=True):
    try:
        doc.ents = [doc.char_span(s["start"], s["end"], s["label"]) for s in eg["spans"]]
        iob_tags = [f"{t.ent_iob_}-{t.ent_type_}" if t.ent_iob_ else "O" for t in doc]
        iob_tags = [t.strip("-") for t in iob_tags] # tags as entity name
        ner_tags = [entities_dict[u] for u in iob_tags] #ner_tags as numbers
        tokens = [str(t) for t in doc]
        temp_data = {
            "id": unique_id, 
            "tokens": tokens,
            "tags": iob_tags,
            "ner_tags": ner_tags
        }
        dataset.append(temp_data)
        unique_id += 1
    except:
        pass

with open('data.jsonl', 'w') as outfile:
    for entry in dataset:
        json.dump(entry, outfile)
        outfile.write('\n')